In [5]:
import cv2

# Load the pre-trained SSD model and configuration file
model_path = 'mobilenet_iter_73000.caffemodel'
config_path = 'deploy.prototxt'
net = cv2.dnn.readNet(model_path, config_path)

# Load the class labels
class_labels = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

# Open a webcam
cap = cv2.VideoCapture(0)  # 0 is the default camera, change it if you have multiple cameras.

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Prepare the frame for object detection
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    # Set the input to the neural network
    net.setInput(blob)

    # Perform object detection
    detections = net.forward()

    # Loop through the detections and draw bounding boxes
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.2:  # Adjust this threshold as needed
            class_id = int(detections[0, 0, i, 1])
            label = class_labels[class_id]
            box = (detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]]).astype(int))

            (startX, startY, endX, endY) = box.astype(int)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()
